<a href="https://colab.research.google.com/github/bird-feeder/BirdFSD-YOLOv5/blob/main/notebooks/BirdFSDV1_YOLOv5_LS_Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [ ]:
! wget -q 'https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-Linux-x86_64.sh'
! bash "Miniforge3-Linux-x86_64.sh" -b -p

In [ ]:
! rm -rf /root/.cache

! git clone https://github.com/bird-feeder/BirdFSD-YOLOv5.git

import shutil
from pathlib import Path
if Path('.env').exists():
    shutil.move('.env', '/content/BirdFSD-YOLOv5/.env')
elif Path('secrets.txt').exists():
    shutil.move('secrets.txt', '/content/BirdFSD-YOLOv5/.env')


%cd /content/BirdFSD-YOLOv5

! /root/miniforge3/bin/python -m pip install -e .
! /root/miniforge3/bin/pip install -r requirements.txt

! mkdir ultralytics
! git clone https://github.com/ultralytics/yolov5.git ultralytics/yolov5

**Send the `.env` file to the notebook:**

- Using [`Croc`](https://github.com/schollz/croc):

  - Run the following command from the root folder where `BirdFSD-YOLOv5` is cloned:

    ```shell
    croc send "BirdFSD-YOLOv5/.env"
    ```

  - Copy the passphrase part and paste it in the cell input below.

- If not using `Croc`:
  - Drag and drop the `.env` file inside the `BirdFSD-YOLOv5` directory in this notebook, and skip the next cell.

In [ ]:
PASSPHRASE = '' #@param {type:"string"}
if not PASSPHRASE:
    raise Exception('Paste the passphrase in the input bar!')
if 'croc' in PASSPHRASE:
    PASSPHRASE = PASSPHRASE.split('croc ')[1]

! curl https://getcroc.schollz.com | bash

! croc --yes $PASSPHRASE

print('\n>>>> Clear the passphrase input field!')

## Download Model Weights

In [ ]:
! /root/miniforge3/bin/python birdfsd_yolov5/model_utils/download_weights.py --model-version latest --output 'best.pt'

## Predict

In [15]:
! /root/miniforge3/bin/python birdfsd_yolov5/prediction/predict.py -h

usage: predict.py [-h] [-w WEIGHTS] -v MODEL_VERSION [-p PROJECT_IDS]
                  [-r TASKS_RANGE] [-a] [-t ONE_TASK] [-m]
                  [--get-tasks-with-api] [-D] [-L IF_EMPTY_APPLY_LABEL] [-d]
                  [-V]

optional arguments:
  -h, --help            show this help message and exit
  -w WEIGHTS, --weights WEIGHTS
                        Path to the model weights
  -v MODEL_VERSION, --model-version MODEL_VERSION
                        Name of the model version
  -p PROJECT_IDS, --project-ids PROJECT_IDS
                        Comma-seperated project ids. If empty, it will select
                        all projects
  -r TASKS_RANGE, --tasks-range TASKS_RANGE
                        Comma-separated range of task ids (e.g., "10,18")
  -a, --predict-all     Predict all tasks even if predictions exist
  -t ONE_TASK, --one-task ONE_TASK
                        Predict a single task
  -m, --multithreading  Enable multithreading
  --get-tasks-with-api  Use label-studio

In [ ]:
#@markdown `MODEL_VERSION` is required!
MODEL_VERSION = '' #@param {'type': 'string'}
#@markdown Leave `LS_PROJECT_IDS` empty to select all projects.
LS_PROJECT_IDS = '' #@param {'type': 'string'}

if not MODEL_VERSION:
    raise Exception('You need to specify a model version!')

! /root/miniforge3/bin/python birdfsd_yolov5/prediction/predict.py \
    --weights 'best.pt' \
    --project-id "$LS_PROJECT_IDS" \
    --model-version "$MODEL_VERSION" \
    --if-empty-apply-label 'background' \
    --predict-all \
    --multithreading